# P. vivax malārijas datu sagatavošana YOLO formātam

Šajā notebook'ā mēs:

- iepazīsim datu struktūru (`data/training` un `data/test`),
- nolasīsim Supervisely/JSON anotācijas,
- pārvērtīsim anotācijas YOLO formātā (`.txt` blakus attēlam),
- izveidosim YOLO datu struktūru (`images/train`, `labels/train`, ...),
- izveidosim `malaria.yaml` konfigurācijas datni YOLO modelim.

Sākotnēji datu struktūra ir:
``` bash
data/
  training/
    img/*.png
    ann/*.json
  test/
    img/*.png
    ann/*.json
```

In [1]:
import json
import os
from pathlib import Path
from typing import Dict, List, Tuple

from PIL import Image
from tqdm import tqdm

In [ ]:
# Saknes ceļi (pielāgo, ja vajag)
DATA_ROOT = Path("data")
ORIG_TRAIN = DATA_ROOT / "training"
ORIG_TEST = DATA_ROOT / "test"

# Kur saglabāsim YOLO-ready versiju
YOLO_ROOT = DATA_ROOT / "malaria_yolo"
IMAGES_TRAIN = YOLO_ROOT / "images" / "train"
IMAGES_VAL = YOLO_ROOT / "images" / "val"
LABELS_TRAIN = YOLO_ROOT / "labels" / "train"
LABELS_VAL = YOLO_ROOT / "labels" / "val"

# Izveidojam jaunas mapes, ja tādu vēl nav
for p in [IMAGES_TRAIN, IMAGES_VAL, LABELS_TRAIN, LABELS_VAL]:
    p.mkdir(parents=True, exist_ok=True)

In [2]:
# Visas iespējamās klases datu kopā (pēc DatasetNinja apraksta)
all_classes = [
    "red blood cell",
    "trophozoite",
    "difficult",
    "ring",
    "schizont",
    "gametocyte",
    "leukocyte",
]

# Mūs interesē tieši malārijas patogēni (inficētās formas)
infected_classes = [
    "trophozoite",
    "ring",
    "schizont",
    "gametocyte",
]

# Izveidojam mapping priekš YOLO klasēm (tikai infekcijām)
class_to_id: Dict[str, int] = {name: i for i, name in enumerate(infected_classes)}

class_to_id

{'trophozoite': 0, 'ring': 1, 'schizont': 2, 'gametocyte': 3}

In [ ]:
def bbox_to_yolo(
    x1: float,
    y1: float,
    x2: float,
    y2: float,
    img_w: int,
    img_h: int,
) -> Tuple[float, float, float, float]:
    """
    Pārveido bbox marķējumu no formāta (x1, y1, x2, y2) pikseļos
    uz YOLO formātu (xc, yc, w, h) normalizētā veidā [0..1] pret attēla augstumu un platumu.
    """
    cx = (x1 + x2) / 2.0
    cy = (y1 + y2) / 2.0
    bw = (x2 - x1)
    bh = (y2 - y1)

    return cx / img_w, cy / img_h, bw / img_w, bh / img_h

In [ ]:
def load_supervisely_annotation(json_path: Path):
    """
    Nolasa vienas anotācijas JSON (Supervisely formāts).
    Atgriež:
      - attēla izmērus (w, h),
      - sarakstu ar (class_title, (x1,y1,x2,y2))
    """
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    img_h = data["size"]["height"]
    img_w = data["size"]["width"]

    boxes = []
    for obj in data["objects"]:
        cls = obj["classTitle"]
        # "points" -> "exterior" satur 2 punktus: [x1,y1], [x2,y2]
        (x1, y1), (x2, y2) = obj["points"]["exterior"]
        boxes.append((cls, (x1, y1, x2, y2)))

    return img_w, img_h, boxes

In [ ]:
def convert_split(
    split_dir: Path,
    images_out_dir: Path,
    labels_out_dir: Path,
    ignore_non_infected: bool = True,
):
    """
    Konvertē vienu splitu (training vai test) uz YOLO formātu.
    - split_dir: piem., data/training
      sagaidām apakšmapes 'img' un 'ann'
    """
    img_dir = split_dir / "img"
    ann_dir = split_dir / "ann"

    img_files = sorted([f for f in img_dir.iterdir() if f.suffix.lower() in [".png", ".jpg", ".jpeg"]])

    print(f"Apstrādājam {split_dir.name}: atrasti {len(img_files)} attēli.")

    for img_path in tqdm(img_files):
        stem = img_path.stem  # bez paplašinājuma
        ann_path = ann_dir / f"{stem}.json"
        if not ann_path.exists():
            # Ja nav anotācijas – izlaižam (vai izmetam brīdinājumu)
            print(f"[BRĪDINĀJUMS] Anotācija nav atrasta: {ann_path}")
            continue

        img_w, img_h, boxes = load_supervisely_annotation(ann_path)

        # Pārkopē attēlu uz YOLO mapi (ja grib, var arī mainīt izmēru)
        target_img_path = images_out_dir / img_path.name
        if not target_img_path.exists():
            # vienkāršākais – nokopē 1:1; šeit var izmēģināt arī .resize, ja grib.
            Image.open(img_path).save(target_img_path)

        # Sagatavojam YOLO anotāciju
        yolo_lines: List[str] = []

        for cls, (x1, y1, x2, y2) in boxes:
            if ignore_non_infected and cls not in infected_classes:
                # Ignorējam RBC, leukocyte, difficult
                continue
            if cls not in class_to_id:
                # gadījumam, ja kāda klase nav mapē
                continue

            cls_id = class_to_id[cls]
            xc, yc, bw, bh = bbox_to_yolo(x1, y1, x2, y2, img_w, img_h)
            yolo_lines.append(f"{cls_id} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}")

        # Saglabājam .txt anotāciju tikai, ja ir vismaz 1 objekts
        if yolo_lines:
            label_path = labels_out_dir / f"{stem}.txt"
            with open(label_path, "w", encoding="utf-8") as f:
                f.write("\n".join(yolo_lines))

In [ ]:
# Training -> YOLO train
convert_split(ORIG_TRAIN, IMAGES_TRAIN, LABELS_TRAIN, ignore_non_infected=True)

# Test -> YOLO val
convert_split(ORIG_TEST, IMAGES_VAL, LABELS_VAL, ignore_non_infected=True)

Pēc datu sagatavošanas būs jauna datu struktūra, kas atbilst YOLO modelim:

``` bash
data/malaria_yolo/
  images/train/*.png
  labels/train/*.txt
  images/val/*.png
  labels/val/*.txt
```

In [ ]:
# Konfigurācijas datnes izveidošana

yaml_text = f"""# YOLO datu konfigurācija P. vivax malārijai
path: {YOLO_ROOT.as_posix()}

train: images/train
val: images/val

names:
"""

for i, name in enumerate(infected_classes):
    yaml_text += f"  {i}: {name}\n"

yaml_path = YOLO_ROOT / "malaria.yaml"
with open(yaml_path, "w", encoding="utf-8") as f:
    f.write(yaml_text)

yaml_path, yaml_text

Konfigurāciajs datni var arī izveidot manuāli, izveidojot YAML datni ar šādu saturu:

``` yaml
# YOLO datu konfigurācija P. vivax malārijai
path: data/malaria_yolo

train: images/train
val: images/val

names:
  0: trophozoite
  1: ring
  2: schizont
  3: gametocyte
```